jupyter-lab --allow-root --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from PIL import Image
import os
import json
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torch.optim as optim
import torchvision.transforms.functional as TF
import torch.nn.functional as F

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_files = os.listdir(self.image_dir)
        self.label_files = os.listdir(self.label_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_filename = self.image_files[idx]
        image_path = os.path.join(self.image_dir, image_filename)
        label_filename = image_filename.replace(".tif", ".json")
        label_path = os.path.join(self.label_dir, label_filename)

        # Load image
        image = Image.open(image_path)

        # Read label file
        with open(label_path, 'r') as f:
            label_data = json.load(f)

        # Extract bounding box coordinates
        boxes = []
        for obj in label_data["objects"]:
            if obj["classTitle"] == "Oil tank":  # Assuming the class name for oil tanks is "Oil tank"
                bbox = obj["points"]["exterior"]
                x_min, y_min = bbox[0]
                x_max, y_max = bbox[1]
                boxes.append([x_min, y_min, x_max, y_max])

        # Convert to tensors
        image_tensor = ToTensor()(image)
        boxes_tensor = torch.tensor(boxes, dtype=torch.float32)

        # Labels are not used in Faster R-CNN, but we include them for consistency
        num_boxes = len(boxes)
        labels_tensor = torch.ones((num_boxes,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes_tensor
        target["labels"] = labels_tensor

        return image_tensor, target


In [ ]:
# Set the directory paths in your Google Drive
# drive_path = "/content/drive/MyDrive/oiltank_dataset"
drive_path = "/Users/crossrunway/DataSets/oiltank_dataset"
image_dir = os.path.join(drive_path, "train_images")
label_dir = os.path.join(drive_path, "train_labels")

In [ ]:
import os
import json

def extract_oil_tank_coordinates(json_data):
    oil_tank_coordinates = []
    features = json_data.get("features", [])
    for feature in features:
        properties = feature.get("properties", {})
        object_imcoords = properties.get("object_imcoords", "")
        coords = [float(coord) for coord in object_imcoords.split(",")]
        oil_tank_coordinates.append(coords)
    return oil_tank_coordinates

# train_labels 파일의 경로
train_labels_path = label_dir

# train_labels 디렉토리 내의 모든 파일 불러오기
json_files = [file for file in os.listdir(train_labels_path) if file.endswith(".json")]

# 각 JSON 파일에서 oil tank의 좌표 출력
for json_file in json_files:
    file_path = os.path.join(train_labels_path, json_file)
    with open(file_path, "r") as file:
        json_data = json.load(file)
    coordinates = extract_oil_tank_coordinates(json_data)

    print(f"Oil Tank Coordinates in {json_file}:")
    for coords in coordinates:
        print(coords)
    print()

Oil Tank Coordinates in OBJ03619_PS3_K3A_NIA0151.json:
[704.1728950442241, 785.0206820712685, 768.8946762486776, 785.0206820712687, 768.8946762486776, 856.6615055690174, 704.1728950442241, 856.6615055690173]
[770.9589241140657, 898.9536206517263, 837.1772336823016, 898.9536206517264, 837.1772336823014, 968.0946024445813, 770.9589241140656, 968.0946024445811]
[591.0451856254213, 852.3591943664937, 657.5328271375024, 852.3591943664937, 657.5328271375024, 920.961512271658, 591.0451856254213, 920.961512271658]
[554.1467093186186, 748.3970640422172, 606.6290897507455, 748.3970640422173, 606.6290897507454, 800.3008014289744, 554.1467093186185, 800.3008014289743]
[610.1677536384359, 716.0772307807841, 662.3808021267174, 716.0772307807841, 662.3808021267173, 768.788963999077, 610.1677536384358, 768.788963999077]
[665.3808021267174, 683.757397519351, 712.2072117380935, 683.7573975193511, 712.2072117380934, 733.7758112991913, 665.3808021267173, 733.7758112991912]
[522.0962080010306, 691.02936000

In [ ]:
# Set the transformation
transform = None  # You can add additional transformations if needed

In [ ]:
# Create the custom dataset
dataset = CustomDataset(image_dir, label_dir, transform=transform)

In [ ]:
# Set batch size and create data loader
batch_size = 16
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Train your Faster R-CNN model using the dataloader
model = fasterrcnn_resnet50_fpn(pretrained=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


/Users/crossrunway/anaconda3/envs/ObjectDetection/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/crossrunway/anaconda3/envs/ObjectDetection/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
# Set the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

In [ ]:
# Set the number of epochs
num_epochs = 10

In [ ]:
def compute_loss(targets, outputs):
    # Extract the predicted labels and bounding boxes from the model outputs
    pred_labels = outputs["labels"]
    pred_boxes = outputs["boxes"]

    # Extract the ground truth labels and bounding boxes from the targets
    gt_labels = targets["labels"]
    gt_boxes = targets["boxes"]

    # Compute the classification loss
    classification_loss = F.cross_entropy(pred_labels, gt_labels)

    # Compute the regression loss
    regression_loss = F.smooth_l1_loss(pred_boxes, gt_boxes)

    # Combine the classification and regression losses
    loss = classification_loss + regression_loss

    return loss

In [ ]:
for epoch in range(num_epochs):
    for images, targets in dataloader:
        images = images.to(device)
        targets = {k: v.to(device) for k, v in targets.items()}

        # Forward pass
        outputs = model(images, targets)

        # Compute the loss
        loss = compute_loss(targets, outputs)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss every few iterations
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

FileNotFoundError: ignored

In [ ]:
import os
import json

# train_labels 파일의 경로
# train_labels_path = "/content/drive/MyDrive/oiltank_dataset/train_labels"
train_labels_path = "/Users/crossrunway/DataSets/oiltank_dataset/train_labels"

# train_labels 디렉토리 내의 모든 파일 불러오기
json_files = [file for file in os.listdir(train_labels_path) if file.endswith(".json")]

# 각 JSON 파일에서 oil tank의 좌표 출력

y_train = [[0] for i in range(len(train_img_list))]

for json_file in json_files:
    file_path = os.path.join(train_labels_path, json_file)
    with open(file_path, "r") as file:
        json_data = json.load(file)
    features = json_data.get("features", [])
    for feature in features:
        properties = feature.get("properties", {})
        image = properties.get("image_id", "")
        for i in range(len(train_img_list)) :
          if train_img_list[i].endswith(image) :
            object_imcoords = list(map(float,properties.get("object_imcoords", "").split(",")))
            y_train[i].append(object_imcoords)
            y_train[i][0] += 1

NameError: ignored

Ver2

In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# 경로 설정
# train_data_path = "/content/drive/MyDrive/oiltank_dataset/train_images_png"
# label_data_path = "/content/drive/MyDrive/oiltank_dataset/train_labels"

train_data_path = "/Users/crossrunway/DataSets/oiltank_dataset/train_images_png"
label_data_path = "/Users/crossrunway/DataSets/oiltank_dataset/train_labels"

# 클래스 개수 설정 (배경 + 실제 클래스 개수)
num_classes = 2  # 배경과 oiltank 두 개의 클래스

In [ ]:
# Faster R-CNN 모델 생성
class FasterRCNN(nn.Module):
    def __init__(self, num_classes):
        super(FasterRCNN, self).__init__()
        # Backbone을 대체하기 위한 간단한 CNN 레이어 정의
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # 객체 감지를 위한 레이어 정의
        self.classifier = nn.Sequential(
            nn.Linear(128 * 64 * 64, 1024),
            nn.ReLU(True),
            nn.Linear(1024, num_classes),
        )

        # 바운딩 박스 regression을 위한 레이어 정의
        self.bbox_regressor = nn.Sequential(
            nn.Linear(128 * 64 * 64, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 4),  # 바운딩 박스의 4개의 좌표(x, y, width, height)를 예측합니다.
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        class_logits = self.classifier(x)
        bbox_deltas = self.bbox_regressor(x)
        return class_logits, bbox_deltas

# 모델 생성
model = FasterRCNN(num_classes=num_classes)

# 모델 학습을 위한 데이터 준비 (학습 데이터와 레이블을 적절하게 로드해야 함)


In [ ]:
# 데이터셋 클래스 생성
class CustomDataset(Dataset):
    def __init__(self, data_path, label_path, transform=None):
        self.data_path = data_path
        self.label_path = label_path
        self.transform = transform
        self.data = os.listdir(data_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data[idx]
        img_path = os.path.join(self.data_path, img_name)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        # 레이블 로드
        label_name = os.path.splitext(img_name)[0] + ".json"
        label_path = os.path.join(self.label_path, label_name)
        with open(label_path, "r") as file:
            json_data = json.load(file)
        features = json_data.get("features", [])
        num_oiltanks = 0
        boxes = []
        for feature in features:
            properties = feature.get("properties", {})
            class_name = properties.get("class_name", "")
            if class_name == "oiltank":
                num_oiltanks += 1
                object_imcoords = list(map(float, properties.get("object_imcoords", "").split(",")))
                boxes.append(object_imcoords)

        return image, {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": torch.ones((num_oiltanks,), dtype=torch.int64),
        }

In [ ]:
# 데이터 전처리 함수 정의 (적절한 전처리가 필요합니다)
def preprocess_image(image):
    # 이미지 크기 조정, 정규화 등의 작업 수행
    # torchvision.transforms.ToTensor()를 사용하여 이미지를 Tensor로 변환
    return ToTensor()(image)

# 학습 데이터 로드
train_dataset = CustomDataset(train_data_path, label_data_path, transform=preprocess_image)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 손실 함수 및 옵티마이저 설정
criterion = torch.nn.CrossEntropyLoss()
bbox_regression_criterion = torch.nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 학습
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for images, targets in train_loader:
        images = images.to(device)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Forward pass
        class_logits, bbox_deltas = model(images)

        # Loss 계산
        class_labels = targets[0]['labels']
        class_loss = criterion(class_logits, class_labels)
        bbox_targets = targets[0]['boxes']
        bbox_regression_loss = bbox_regression_criterion(bbox_deltas, bbox_targets)

        loss = class_loss + bbox_regression_loss

        # Backward pass 및 옵티마이저 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')

# 학습된 모델 저장
torch.save(model.state_dict(), 'faster_rcnn_model.pth')

AttributeError: ignored